### Table of Contents

* [Relevant libraries and colour dictionaries](#chapter1)
* [SpatialDE](#chapter2)
* [Top30](#chapter3)
    * [Top 30 compare](#section_3_1)
* [Scatterplot](#chapter4)
* [AEH](#chapter5)

#### Relevant libraries and colour dictionaries <a class="anchor" id="chapter1"></a>

In [1]:
# Import relevant libraries
import numpy as np
import scanpy as sc
import os
import pandas as pd
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns
from collections import OrderedDict
from matplotlib import cm
import anndata as ann
import scanpy.external as sce
from datetime import datetime
import NaiveDE
import SpatialDE
from matplotlib_venn import venn3
%matplotlib inline

# Set current directory
os.chdir("/Users/mendenlab/work/spatial_granuloma/scripts")

# assign the rigth colours to the right annotation
def _set_colors(adata, obs_name, colors):
    """Set palette with specific colors for specific categories

    Parameters
    ----------
    adata : annData
    obs_name : column to plot
    colors : OrderedDict(): colors named by categories

    Returns
    -------

    """
    if len(colors.values())>0:
        palette = []
        unique_colors = np.unique(adata.obs[obs_name])
        for key in adata.obs[obs_name].cat.categories.tolist():
            if key in colors.keys():
                palette.append(colors[key])
    return palette

#Set the colours per annotation
spot_colors = []
spot_colors = OrderedDict()
spot_colors["EPIDERMIS"] = 'blue'
spot_colors["DERMIS"] = '#E0EEE0'
spot_colors["INTERFACE"] = 'deepskyblue'
spot_colors["VESSEL"] = 'darkgreen'
spot_colors["HAIR FOLLICLE"] = "#543005"
spot_colors["SWEAT GLAND"] = 'y'
spot_colors["SEBACEOUS GLAND"] = 'mistyrose'
spot_colors["MUSCLE"] = 'darkcyan'
spot_colors["GA"] = 'firebrick'  
spot_colors["GNL"] = 'orchid'
spot_colors["GSS"] = 'blueviolet'
spot_colors["GSC"] = 'mediumvioletred'
spot_colors["UNDETERMINED"] = 'black'


dermis_colors = []
dermis_colors = OrderedDict()
dermis_colors["UNDETERMINED"] = 'black'
dermis_colors["upper EPIDERMIS"] = 'blue'
dermis_colors["middle EPIDERMIS"] = 'dodgerblue'
dermis_colors["basal EPIDERMIS"] = 'skyblue'
dermis_colors["DERdepth1"] = '#006837'
dermis_colors["DERdepth2"] = '#238443'
dermis_colors["DERdepth3"] = '#41AB5D'
dermis_colors["DERdepth4"] = '#78C679'
dermis_colors["DERdepth5"] = '#ADDD8E'
dermis_colors["DERdepth6"] = '#D9F0A3'
dermis_colors["DERdepth7"] = '#F7FCB9'

leiden_r13_colours = []
leiden_r13_colours = OrderedDict()
leiden_r13_colours["0"] = 'darkolivegreen'
leiden_r13_colours["1"] = "#D9F0A3"
leiden_r13_colours["2"] = '#238443'
leiden_r13_colours["3"] = 'firebrick'
leiden_r13_colours["4"] = '#78C679'
leiden_r13_colours["5"] = '#78C679'
leiden_r13_colours["6"] = '#41AB5D'
leiden_r13_colours["7"] = '#006837'
leiden_r13_colours["8"] = '#ADDD8E'
leiden_r13_colours["9"] = "#238443"
leiden_r13_colours["10"] = '#78C679'
leiden_r13_colours["11"] = 'blue'
leiden_r13_colours["12"] = 'orchid'
leiden_r13_colours["13"] = '#F46D43'
leiden_r13_colours["14"] = 'dodgerblue'
leiden_r13_colours["15"] = 'deepskyblue'
leiden_r13_colours["16"] = '#cfafaf'
leiden_r13_colours["17"] = 'yellow'
leiden_r13_colours["18"] = 'darkcyan'
leiden_r13_colours["19"] = '#006837'


In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [3]:
# Import adata 
adata_path = "../results/current/"

adata = sc.read(os.path.join(adata_path, "final/Granuloma_QC_clustering.h5"))
    
# setting up "factors" with different levels, order = TRUE
# add less common annotations LAST so they are not overwritten

# Set spot_type and skin_layer as categories and define the levels in each category
# Spot type: annatomical annotations
adata.obs['spot_type'] = pd.Categorical(
    adata.obs['spot_type'],
    categories = ['UNDETERMINED', 'DERMIS', "EPIDERMIS", 'INTERFACE', 'HAIR FOLLICLE',
                'VESSEL', 'MUSCLE', 'SEBACEOUS GLAND', 'SWEAT GLAND', 'GA', 'GNL', 'GSS', 'GSC'],
                 ordered = True)

# Skin layer
adata.obs['skin_layer'] = pd.Categorical(
    adata.obs['skin_layer'],
    categories = ['UNDETERMINED', 
                'upper EPIDERMIS', 'middle EPIDERMIS', 'basal EPIDERMIS',
                'DERdepth1', 'DERdepth2', 'DERdepth3', 'DERdepth4',
                'DERdepth5', 'DERdepth6', 'DERdepth7'],
    ordered = True)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [4]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/SpatialDE/") # Set working directory so it saves it in the drive
print(os.getcwd())

/Volumes/Drive/spatial_granuloma/output/SpatialDE


In [ ]:
# os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
# print('Saving plots in: ' + os.getcwd())
# spatialde_path = "../output/SpatialDE/"

# date = datetime.now().strftime("%Y%m%d") #For timestamp in file names

# sample_SPECIMEN_list = adata.obs['sample_SPECIMEN'].unique()

# fig, axes = plt.subplots(facecolor='w', edgecolor='k', figsize=(15, 80), dpi=300)
# for i, sample_SPECIMEN in enumerate(sample_SPECIMEN_list):
#     adata_subset = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)]
#     ax = fig.add_subplot(22, 2, i + 1)
    
#     if ((adata_subset.obs['sample'].unique() == 'P17851_1002') | (adata_subset.obs['sample'].unique() == 'P18554_1006') | (adata_subset.obs['sample'].unique() == 'P18554_1002')):
#         ax.invert_xaxis() 
#         ax.invert_yaxis()
#     plt.scatter(adata_subset.obs['array_col'], 
#                 -1.5*adata_subset.obs['array_row'],
#                 c = adata_subset.obs['spot_type'].map(spot_colors))
#     plt.title(sample_SPECIMEN)
#     plt.colorbar(ticks=[]);
# plt.savefig(spatialde_path + date + '_Separate_slides_colouredbyspottype.png')

#### SpatialDE <a class="anchor" id="chapter2"></a>

In [5]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/GA_slides/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220511' # timestamp of saved files, check if they are the most recent in folder
# Loop
# -------------------------------------------------------------------------------------------------------
for sample_SPECIMEN in sample_SPECIMEN_list:
    
    print('Starting analysis for: ' + sample_SPECIMEN)
    adata_subset = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    
    # Get the counts dataframe
    counts = adata_subset.to_df(layer="counts")
    counts = counts.T[counts.sum(0) >= 3].T  # Filter practically unobserved genes
    print(counts.shape)
    
    # Correct for library size or sequencing depth
    sample_info = sc.get.obs_df(adata_subset, keys=['array_row', 'array_col', "n_counts"])
    counts = counts.loc[sample_info.index]  # Align count matrix with metadata table
    print(sample_info.head(5))
    norm_expr = NaiveDE.stabilize(counts.T).T
    resid_expr = NaiveDE.regress_out(sample_info, norm_expr.T, 'np.log(n_counts)').T
    
    # Save all the data
    sample_info.to_csv(spatialde_path + date + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts.to_csv(spatialde_path + date + '_counts_' + sample_SPECIMEN + '.csv')
    sc.write(os.path.join(spatialde_path, date + '_adata_subset_' + sample_SPECIMEN + '.h5'), adata_subset)
    norm_expr.to_csv(spatialde_path + date + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr.to_csv(spatialde_path + date + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Run Spatial DEG
    print('Saving plot as: ' + spatialde_path + date + sample_SPECIMEN + '.csv')
    if not os.path.isfile(os.path.join(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')):
        print('Calculating spatial DE for patient 1 slide 1')
        X = sample_info[['array_row', 'array_col']]
        results = SpatialDE.run(X, resid_expr)
        results.to_csv(spatialde_path + date + '_results_' + sample_SPECIMEN + '.csv')
        print('Wohoo!' + sample_SPECIMEN + ' data saved!')
        print("")
    
    # or load it if already computed
    else:
        print('Spatial DE already computed for ' + sample_SPECIMEN + ', importing results . csv file from: ' + spatialde_path)
        print('Need to load one by one, or write new code to load them all!:)')


Saving plots in: ../output/SpatialDE/GA_slides/
Starting analysis for: P17851_1001_91253-A
(1014, 13406)
                  array_row  array_col  n_counts
AAACGAAGAACATACC          6         64    4625.0
AAAGACCCAAGTCGCG         10         48    1079.0
AAAGGGATGTAGCAAG         24         62    1040.0
AAAGGGCAGCTTGAAT         24         26     673.0
AAAGTCACTGATGTAA         10         52     968.0


/Users/mendenlab/miniconda3/envs/r4.1.2_py3.8/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


KeyboardInterrupt: 

In [ ]:
# Visualise spatial slides
for sample_SPECIMEN in sample_SPECIMEN_list:
    adata_subset = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    fig,axes = plt.subplots(facecolor = 'w', edgecolor = 'k', figsize = (40, 20))
    
    ax = plt.subplot(3, 3, 1)
    if ((adata_subset.obs['sample'].unique() == 'P17851_1002') | (adata_subset.obs['sample'].unique() == 'P18554_1006') | (adata_subset.obs['sample'].unique() == 'P18554_1002')):
        ax.invert_xaxis() 
        ax.invert_yaxis()
    plt.title('coloured by spot type', fontsize = 20)
    plt.axis('equal')
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['spot_type'].map(spot_colors))
    
    ax = plt.subplot(3, 3, 2)
    if ((adata_subset.obs['sample'].unique() == 'P17851_1002') | (adata_subset.obs['sample'].unique() == 'P18554_1006') | (adata_subset.obs['sample'].unique() == 'P18554_1002')):
        ax.invert_xaxis() 
        ax.invert_yaxis()
    plt.title(sample_SPECIMEN + '\n coloured by skin layer', fontsize = 20)
    plt.axis('equal')
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['skin_layer'].map(dermis_colors))
    
    ax = plt.subplot(3, 3, 3)
    if ((adata_subset.obs['sample'].unique() == 'P17851_1002') | (adata_subset.obs['sample'].unique() == 'P18554_1006') | (adata_subset.obs['sample'].unique() == 'P18554_1002')):
        ax.invert_xaxis() 
        ax.invert_yaxis()
    plt.axis('equal')
    plt.title('coloured by leiden clusters (r = 1.3)', fontsize = 20)
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['leiden_r1.3_patient'].map(leiden_r13_colours))
    plt.axis('equal')
    plt.colorbar(ticks=[]);
    plt.savefig(spatialde_path + date + '_Slides_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 120)
    plt.show()

#### Top 30 spatially differentially expressed genes <a class="anchor" id="chapter3"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/GA_slides/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']

timestamp = '20220511' # the timestamp of the files (check filename)

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_path + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_path + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_path + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_path + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Print results
    print(results.sort_values('qval').head(10)[['g', 'l', 'qval']])

    # Genes - top 30 by qval
    genes = results.sort_values('qval').head(30)['g'].tolist()
    textfile = open(spatialde_path + 'Top30/' + date + '_top30genes_' + sample_SPECIMEN + '.txt', "w")
    for gene in genes:
        textfile.write(gene + "\n")
    textfile.close()
    
    # Plot genes in spatial slides - top 30 by qval
    print('Plotting genes in spatial slides!')
    fig, axes = plt.subplots(facecolor='w', edgecolor='k', figsize=(20, 80))
    for i, g in enumerate(genes):
        ax = plt.subplot(15, 2, 1)
        if ((adata_sample_SPECIMEN.obs['sample'].unique() == 'P17851_1002') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1006') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1002')):
            ax.invert_xaxis() 
            ax.invert_yaxis()
        plt.scatter(adata_sample_SPECIMEN.obs['array_col'], -1.5*adata_sample_SPECIMEN.obs['array_row'], c = adata_sample_SPECIMEN.obs['spot_type'].map(spot_colors))
        plt.axis('equal')
        
        ax = plt.subplot(15, 2, i + 1)
        if ((adata_sample_SPECIMEN.obs['sample'].unique() == 'P17851_1002') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1006') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1002')):
            ax.invert_xaxis() 
            ax.invert_yaxis()
        plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c=norm_expr[g], s = 80);
        plt.title(g, fontsize=20)
        plt.axis('equal')
        plt.colorbar(ticks=[]);
    plt.savefig(spatialde_path + 'Top30/' + date + '_Top30qval_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 120)
    plt.show()
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)
    

#### Top 30 compare  <a class="anchor" id="section_3_1"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/GA_slides/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']

timestamp = '20220511' # the timestamp of the files (check filename)

# Loop to read genes .txt file
# -------------------------------------------------------------------------------------------------------
all_top30genes = {}
for sample_SPECIMEN in sample_SPECIMEN_list:
    file_path = spatialde_path + 'Top30/' + timestamp + '_top30genes_' + sample_SPECIMEN + '.txt'
    
    with open(file_path) as f:
        all_top30genes[sample_SPECIMEN] = f.read().splitlines()
    
#print(all_top30genes)
df = pd.DataFrame(all_top30genes.items(), columns = ['sample_SPECIMEN', 'genes']).explode('genes')
df.genes.value_counts() > 1
v = df.genes.value_counts()
df_filtered = df[df.genes.isin(v.index[v.gt(1)])]
#df['genes'].value_counts().plot(kind='bar')

# Plot common genes
# -------------------------------------------------------------------------------------------------------
df_plot = df_filtered.groupby(['sample_SPECIMEN', 'genes']).size().reset_index().pivot(columns='sample_SPECIMEN', index='genes', values=0)
df_plot

plt.figure(facecolor='w', edgecolor='k', figsize=(60, 60))
df_plot.plot.bar(stacked = True, figsize=(15, 8),
                 color = ['dodgerblue', 'deepskyblue', 'lightblue', 'orange', 'gold', 'purple', 'fuchsia', '#8FBC8F', '#C1FFC1', '#8B0000', '#FF0000'])
plt.title("Common top 30 spatially-DE genes \n in granuloma spots (GA)")
plt.xlabel("Common genes")
plt.ylabel("Count")
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.75))
plt.savefig(spatialde_path + 'Top30/' + date + '_Barplot_common_top30' + '.png', bbox_inches='tight', dpi = 120)

#### Scatterplots <a class="anchor" id="chapter4"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/GA_slides/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']

timestamp = '20220511' # the timestamp of the files (check filename)

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()

    # Load the data for the particular sample
    results = pd.read_csv(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_path + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
#     sample_info = pd.read_csv(spatialde_path + 'SpatialDE_Zoom/' + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
#     counts = pd.read_csv(spatialde_path + 'SpatialDE_Zoom/' + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
#     resid_expr = pd.read_csv(spatialde_path + 'SpatialDE_Zoom/' + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Highlight gene groups
    group1 = results.sort_values('FSV').tail(3)[['g', 'l', 'qval', 'FSV']] # highest FSV
    print(group1)
    group2 = results.sort_values('qval').head(3)[['g', 'l', 'qval', 'FSV']] # lowest q-value
    print(group2)    
    group3 = results[(results.qval > 1e-10) & (results.FSV < 0.98)].sort_values('FSV').tail(3)[['g', 'l', 'qval', 'FSV']]
    print(group3)
    group4 = results[(results.qval > 1e-15) & (results.FSV < 0.9)].sort_values(['qval', 'FSV'], ascending = [True, False]).head(3)[['g', 'l', 'qval', 'FSV']]
    print(group4)
    group5 = results[(results.qval < 1e-2) & (results.qval > 1e-4) & (results.FSV < 0.9)].sort_values(['FSV', 'qval'], ascending = [False, True]).head(3)[['g', 'l', 'qval', 'FSV']]
    print(group5)
    
    # Plot scatterplot
    print('Plotting scatterplot!')
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(10, 5))
    plt.yscale('log')
    plt.title(sample_SPECIMEN)
    plt.scatter(results['FSV'], 
                results['qval'] + 1e-20, 
                c = 'lightgray')
#     plt.scatter(results.loc[results['g'].isin(group1.g)]['FSV'], 
#                 results.loc[results['g'].isin(group1.g)]['qval'] + 1e-20, # we have to sum 1e-18 to solve the problem of log(0)
#                 c = 'red')
#     plt.scatter(results.loc[results['g'].isin(group2.g)]['FSV'], 
#                 results.loc[results['g'].isin(group2.g)]['qval'] + 1e-20, 
#                 c = 'purple')
#     plt.scatter(results.loc[results['g'].isin(group3.g)]['FSV'], 
#                 results.loc[results['g'].isin(group3.g)]['qval'] + 1e-20, 
#                 c = 'green')
#     plt.scatter(results.loc[results['g'].isin(group4.g)]['FSV'], 
#                 results.loc[results['g'].isin(group4.g)]['qval'] + 1e-20, 
#                 c = 'orange')
#     plt.scatter(results.loc[results['g'].isin(group5.g)]['FSV'], 
#                 results.loc[results['g'].isin(group5.g)]['qval'] + 1e-20, 
#                 c = 'blue')

    plt.axhline(0.05, c='black', lw=1, ls='--')
    plt.axhline(0.01, c='black', lw=1, ls='--')
    
    plt.gca().invert_yaxis();
    plt.xlabel('Fraction spatial variance')
    plt.ylabel('Adj. P-value');
    plt.savefig(spatialde_path + 'Scatterplots/' + date + '_Scatterplot_' + sample_SPECIMEN + '.png')
    plt.show()
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

In [ ]:
patterns.head()

In [ ]:
patterns.head()
plt.scatter(95*sample_info['array_col'], 
            -1.5*sample_info['array_row'],
            c = patterns[0])
plt.colorbar(ticks=[]);

#### AEH <a class="anchor" id="chapter5"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/GA_slides/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GA spots
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']

timestamp = '20220511' # the timestamp of the files (check filename)
lengthscale_vector = [1]
conditions = ['qval < 0.01']
number_of_patterns = 8

# Loop for each sample_SPECIMEN
# -------------------------------------------------------------------------------------------------------

plot_only = True # If false then we will compute AEH from scratch 

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()

    # Load the data for the particular sample
    results = pd.read_csv(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_path + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_path + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_path + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_path + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    X = sample_info[['array_row', 'array_col']]
    
    # Loop for AEH
    for cond in conditions:
        print('Subsetting results for condition: ' + cond)
        print('')
        sign_results = results.query(cond)
        
        if plot_only == False:
            print('Saving df as: ' + spatialde_path + 'AEH/' + date + '_sign_results_' + cond + '.csv')
            sign_results.to_csv(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_sign_results_' + cond + '.csv')

        for lengthscale in lengthscale_vector:
            if (plot_only == False):
                print('calculating spatial de for lengthscale = ' + str(lengthscale))
                print('')
                histology_results, patterns = SpatialDE.aeh.spatial_patterns(X, resid_expr, sign_results, C=number_of_patterns, 
                                                                             l=lengthscale, verbosity=1)
                print('Saving spatial DE results')
                print('')
                patterns.to_csv(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
                histology_results.to_csv(spatialde_path + 'AEH/' + date + '_' +  sample_SPECIMEN + '_histology_results_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
            else:
                patterns = pd.read_csv(spatialde_path + 'AEH/' + timestamp + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.csv', index_col=0)
                histology_results = pd.read_csv(spatialde_path + 'AEH/' + timestamp + '_' +  sample_SPECIMEN + '_histology_results_' + cond + '_lengthscale' + str(lengthscale) + '.csv', index_col=0)
            
            print('Computing plots')
            print('')
            fig,axes = plt.subplots(facecolor='w', edgecolor='k', figsize=(20, int(3*number_of_patterns)))
            for i in range(number_of_patterns):
                fig.suptitle(cond + '_lengthscale' + str(lengthscale), fontsize=22, y = 0.9)
                ax = plt.subplot(int(number_of_patterns/2), 2, i + 1)
                if ((adata_sample_SPECIMEN.obs['sample'].unique() == 'P17851_1002') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1006') | (adata_sample_SPECIMEN.obs['sample'].unique() == 'P18554_1002')):
                    ax.invert_xaxis() 
                    ax.invert_yaxis()
                plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c = patterns[str(i)], s = 80); # Note: if patterns.csv is read in, you might need to change it to patterns[str(i)] 
                plt.axis('equal')
                plt.title('Pattern {} - {} genes'.format(i, histology_results.query('pattern == @i').shape[0] ))
                plt.colorbar(ticks=[]);
                plt.savefig(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.png', bbox_inches='tight', dpi = 120)
            print('Patterns successfully plotted and saved!')
            print('')
            
    print('Wohoo! AEH finished for: ' + sample_SPECIMEN + 'condition ' + cond)
    